# Getting starting fine-tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [12]:
%cd /home/arresejo
!git clone https://github.com/mistralai/mistral-finetune.git

/home/arresejo


Install all required dependencies:

In [22]:
!pip install -r /home/arresejo/mistral-finetune/requirements.txt

Defaulting to user installation because normal site-packages is not writeable


## Model download

In [1]:
!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2024-05-30 17:38:57--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.6.117, 104.26.7.117, 172.67.70.68, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.6.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  54.0MB/s    in 5m 29s  

2024-05-30 17:44:26 (42.0 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [14]:
!DIR=/home/arresejo/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [ ]:
# Alternatively, you can download the model from Hugging Face

# !pip install huggingface_hub
# from huggingface_hub import snapshot_download
# from pathlib import Path

# mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
# mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

#! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
#! rm -r /root/mistral_models/7B-v0.3

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-v0.3'

In [15]:
!ls /home/arresejo/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [18]:
# navigate to this data directory
%cd /home/arresejo/data

/home/arresejo/data


In [31]:
!ls /home/arresejo/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [33]:
# navigate to the mistral-finetune directory
%cd /home/arresejo/mistral-finetune/

/home/arresejo/mistral-finetune


In [44]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python3 -m utils.validate_data --train_yaml example/7B.yaml


0it [00:00, ?it/s]Validating /home/arresejo/data/chip8_chunk_train.jsonl ...

100%|█████████████████████████████████| 193040/193040 [02:50<00:00, 1133.98it/s]
1it [02:50, 170.48s/it]
No errors! Data is correctly formated!
Stats for /home/arresejo/data/chip8_chunk_train.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:33:38",
        "data_tokens": 122770610,
        "train_tokens": 78643200,
        "epochs": "0.64",
        "max_steps": 300,
        "data_tokens_per_dataset": {
            "/home/arresejo/data/chip8_chunk_train.jsonl": "122770610.0"
        },
        "train_tokens_per_dataset": {
            "/home/arresejo/data/chip8_chunk_train.jsonl": "78643200.0"
        },
        "epochs_per_dataset": {
            "/home/arresejo/data/chip8_chunk_train.jsonl": "0.6"
        }
    }
}
0it [00:00, ?it/s]Validating /home/arresejo/data/chip8_chunk_eval.jsonl ...

100%|███████████████████████████████████| 10160/10160 [00:07<00:00, 1292.90it/s]
1it [00:07,  7.89

## Start training

In [48]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [46]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/home/arresejo/data/chip8_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/home/arresejo/data/chip8_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/home/arresejo/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence lentgh of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/home/arresejo/run"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [21]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [60]:
import torch

torch.cuda.is_available()

False

In [61]:
import torch
print(torch.version.cuda)

12.1


In [50]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

/home/arresejo/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
args: TrainArgs(data=DataArgs(data='', shuffle=False, instruct_data='/home/arresejo/data/chip8_chunk_train.jsonl', eval_instruct_data='/home/arresejo/data/chip8_chunk_eval.jsonl', instruct=InstructArgs(shuffle=True, dynamic_chunk_fn_call=True)), model_id_or_path='/home/arresejo/mistral_models', run_dir='/home/arresejo/run', optim=OptimArgs(lr=0.0001, weight_decay=0.1, pct_start=0.05), seed=0, num_microbatches=8, seq_len=8192, batch_size=1, max_norm=1.0, max_steps=100, log_freq=1, ckpt_freq=100, save_adapters=True, no_ckpt=False, num_ckpt_keep=3, eval_freq=100, no_eval=False, checkpoint=True, world_size=1, wandb=WandbArgs(project=None, offline=False, key=None, run_name=None), mlflow=MLFlowArgs(tracking_uri=

## Inference

In [24]:
!pip install mistral_inference

In [25]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Machine learning is a subset of artificial intelligence that involves the use of algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of computer science that enables machines to learn and improve from experience without being explicitly programmed. Machine learning algorithms can learn from data and make predictions or decisions based
